In [1]:
using Logging
global_logger(ConsoleLogger());

In [2]:
include("Ratio_of_pseudo_gradient_and_gradient.jl");

Set parameter Username
Academic license - for non-commercial use only - expires 2023-11-06


In [3]:
## Parameters
μ = 0.5
L = 1
R = 1 # min(R_term_1,R_term_2)
bigM = 1e6
η_2 = 1 # should be 1 for PRP, 0 for FR
c_0_input_initial_given = 2

ϵ_tol_feas_lazy_eigval = 1e-1
ϵ_tol_feas_Gurobi = 1e-2
ϵ_tol_feas_input = 1e-8

## Parameter selection to compute ||d_k||^2/||g_k||^2
c_f_input, c_x_input, c_g_input, c_p_input =  0, 0, 0, 1
ι_f_input, ι_x_input, ι_g_input, ι_p_input = 0, 0, 1, 0;

In [4]:
G_ws, Ft_ws, L_cholesky_ws, γ_kM1_ws, β_kM1_ws = feasible_sol_generator_Lyapunov_Ada_PEP(μ, L, R, η_2; c_f = c_f_input, c_x = c_x_input, c_g = c_g_input, c_p = c_p_input, ι_f = ι_f_input, ι_x = ι_x_input, ι_g = ι_g_input, ι_p = ι_p_input)

k = 1;
⋆ = -1;
I_N_star = [⋆; k-1; k]
idx_set_λ_ws_effective = index_set_constructor_for_dual_vars_full(I_N_star);

[ Info: creating initial condition for pseudograd/grad ratio
[ Info: conjugacy condition is satisfied


In [5]:
## Generate locally optimal solution

G_loc_opt, Ft_loc_opt, L_cholesky_loc_opt, γ_kM1_loc_opt, β_kM1_loc_opt, t_loc_opt, lower_bound_var_loc_opt, upper_bound_var_loc_opt, solve_time_loc_opt = Ratio_of_Pseudograd_and_Grad_Model(
    # different parameters to be used
    # -------------------------------
    μ, L, R,
    # warm-start points
    # -----------------
    idx_set_λ_ws_effective, G_ws, Ft_ws, L_cholesky_ws, γ_kM1_ws,  β_kM1_ws;
    # different options
    # -----------------
    # [🐘 ] solver type
    solver = LocalOptSolver,
    # the solvers are:
    # GlobalOptSolver to find the globally optimal solution (uses gurobi)
    # LocalOptSolver to find the locally optimal solution (uses KNITRO)
    # BoundCompSolver to find valid bounds on the decision variables (uses Alpine)
    # [🐯 ] cost coefficients
    c_f = c_f_input, c_x = c_x_input, c_g = c_g_input, c_p = c_p_input,
    ι_f = ι_f_input, ι_x = ι_x_input, ι_g = ι_g_input, ι_p = ι_p_input,
    show_output = :on, # options are :on and :off
    lower_bound_opt_val = 0,
    upper_bound_opt_val = Inf,
    impose_pattern_interpolation = :off,
    impose_pattern_noninterpolation = :off,
    bound_impose = :off, # # options are :generic, and :off
    bound_M = Inf,
    lower_bound_var = [], # lower bound vector on the decision variables
    upper_bound_var = [], # upper bound vector on the decision variables
    PSDness_modeling = :exact, # options are :exact and :through_ϵ and :lazy_constraint_callback
    ϵ_tol_feas = 1e-6, # feasiblity tolerance for minimum eigenvalue of G
    maxCutCount=1e6, # number of lazy cuts when we use lazy constraints for modeling G = L_cholesky*L_cholesky
    PRP_Plus = :off, # options are :positive_case, :negative_case and :off
    c_0_input_initial = c_0_input_initial_given
    )

[ Info: [🎉 ] defining the variables
[ Info: no bound imposed


[ Info: [🎍 ] adding the data generator function to create 𝐱, 𝐠, 𝐟
[ Info: [🎢 ] adding the constraints


[ Info: coninterpol ℓ = i_j_idx(-1, 0)
[ Info: coninterpol ℓ = i_j_idx(-1, 1)


[ Info: coninterpol ℓ = i_j_idx(0, -1)
[ Info: coninterpol ℓ = i_j_idx(0, 1)
[ Info: coninterpol ℓ = i_j_idx(1, -1)
[ Info: coninterpol ℓ = i_j_idx(1, 0)


[ Info: [🎇 ] adding objective
[ Info: [👲 ] warm-start values for all the variables


[ Info: [🙌 	🙏 ] model building done, starting the optimization process
[ Info: [🎴 ] Analyzing PRP/FR method



           Academic License
       (NOT FOR COMMERCIAL USE)
         Artelys Knitro 13.1.0

Knitro presolve eliminated 80 variables and 142 constraints.

algorithm:               1
convex:                  0
datacheck:               0
hessian_no_f:            1
honorbnds:               1
strat_warm_start:        1
The problem is identified as a QCQP.
Knitro shifted start point further inside presolved bounds (4 variables).
Knitro fixing 78 variables eliminated from the presolve.

Problem Characteristics                                 (   Presolved)
-----------------------
Objective goal:  Maximize
Objective type:  linear
Number of variables:                                111 (          31)
    bounded below only:                               2 (           9)
    bounded above only:                               0 (           0)
    bounded below and above:                          0 (           1)
    fixed:                                            6 (           0)
    free:     

[ Info: [😻 ] optimal solution found done, store the solution
[ Info: [💹 ] warm-start objective value = 0.02235169480723308, and objective value of found solution = 1.1249999695663553


[ Info: IMPORTANT INFO
[ Info: ------------------------------------------
[ Info: ||g_{k-1}||^2 =  1.0916394762538901
[ Info: ||p_{k-1}||^2 = 2.183278952509041
[ Info: NCG-PEP Theory {||p_k||^2/||g_k||^2} = 1.125
[ Info: NCG-PEP Numerical {||p_k||^2/||g_k||^2}= 1.1249999695663553
[ Info: ------------------------------------------
[ Info: [💀 ] The dual multipliers for interpolation inequalities are: [-3.33529149126806e-7, -5.365004931689542e-8, -7.483988387091643e-8, -0.9160526921219829, -3.4531124975106654e-7, -0.9160523442979318]


([3.5242248760390797 1.95703488587636 1.3993761399899427 1.8463623918617562; 1.95703488587636 1.0916394762538901 0.7387961408053838 1.0916394762508475; 1.3993761399899427 0.7387961408053838 1.0 0.0; 1.8463623918617562 1.0916394762508475 0.0 2.183278952509041], [1.0303357359923555, 0.6209709436660351], [1.8772918995294978 0.0 0.0 0.0; 1.0424776702902956 0.06985544502760632 0.0 0.0; 0.745422776468945 -0.5481384962067953 0.37932713229807435 0.0; 0.9835244014660188 0.9496360593032902 -0.5604909073797449 0.0], 0.7499999793559416, 0.2392766704360783, 1.1249999695663553, Any[], Any[], 0.014503000304102898)

In [6]:
## Set the warm-starting points for the later stage

G_ws, Ft_ws, L_cholesky_ws, γ_kM1_ws, β_kM1_ws, t_ws, lower_bound_var_ws, upper_bound_var_ws, solve_time_ws = G_loc_opt, Ft_loc_opt, L_cholesky_loc_opt, γ_kM1_loc_opt, β_kM1_loc_opt, t_loc_opt, lower_bound_var_loc_opt, upper_bound_var_loc_opt, solve_time_loc_opt

bound_M_G_loc_opt = 3*maximum(G_loc_opt[i,i] for i in 1:size(G_loc_opt,1))

bound_M_F_loc_opt = 3*maximum(Ft_loc_opt[i] for i in 1:size(Ft_loc_opt,1))

bound_M_loc_opt = maximum([bound_M_G_loc_opt, bound_M_F_loc_opt])	

10.57267462811724

In [8]:
## Compute globally optimal solution

G_glb_opt, Ft_glb_opt, L_cholesky_glb_opt, γ_kM1_glb_opt, β_kM1_glb_opt, t_glb_opt, lower_bound_var_glb_opt, upper_bound_var_glb_opt, solve_time_glb_opt = Ratio_of_Pseudograd_and_Grad_Model(
	# different parameters to be used
    # -------------------------------
    μ, L, R,
    # warm-start points
    # -----------------
    idx_set_λ_ws_effective, G_ws, Ft_ws, L_cholesky_ws, γ_kM1_ws,  β_kM1_ws;
	# different options
	# -----------------
	# [🐘 ] solver type
	solver = GlobalOptSolver,
	# the solvers are:
	# GlobalOptSolver to find the globally optimal solution (uses gurobi)
	# LocalOptSolver to find the locally optimal solution (uses KNITRO)
	# BoundCompSolver to find valid bounds on the decision variables (uses Alpine)
	# [🐯 ] cost coefficients
    c_f = c_f_input, c_x = c_x_input, c_g = c_g_input, c_p = c_p_input,
	ι_f = ι_f_input, ι_x = ι_x_input, ι_g = ι_g_input, ι_p = ι_p_input,
	show_output = :on, # options are :on and :off
	lower_bound_opt_val = 0,
	upper_bound_opt_val = Inf,
	impose_pattern_interpolation = :on,
	impose_pattern_noninterpolation = :off,
	bound_impose = :generic, # # options are :generic, and :off
	bound_M = bound_M_loc_opt,
	lower_bound_var = [], # lower bound vector on the decision variables
	upper_bound_var = [], # upper bound vector on the decision variables
	PSDness_modeling = :lazy_constraint_callback, # options are :exact and :through_ϵ and :lazy_constraint_callback
    ϵ_tol_feas = 1e-6, # feasiblity tolerance for minimum eigenvalue of G
    maxCutCount=1e6, # number of lazy cuts when we use lazy constraints for modeling G = L_cholesky*L_cholesky
	PRP_Plus = :off, # options are :positive_case, :negative_case and :off
	c_0_input_initial = c_0_input_initial_given
	)

Set parameter NonConvex to value 2
Set parameter MIPFocus to value 3
Set parameter MIPGap to value 0.01
Set parameter Presolve to value 2
Set parameter MIPFocus to value 1
Set parameter NonConvex to value 2
Set parameter MIPFocus to value 1
Set parameter MIPGap to value 0.01
Set parameter Presolve to value 2
Set parameter LazyConstraints to value 1
Gurobi Optimizer version 10.0.0 build v10.0.0rc2 (mac64[rosetta2])

CPU model: Apple M1 Max
Thread count: 10 physical cores, 10 logical processors, using up to 10 threads

Optimize a model with 146 rows, 71 columns and 978 nonzeros
Model fingerprint: 0xc52c8a94
Model has 45 quadratic constraints
Coefficient statistics:
  Matrix range     [1e-13, 4e+00]
  QMatrix range    [5e-01, 2e+00]
  QLMatrix range   [1e+00, 2e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [2e+00, 1e+01]
  RHS range        [1e+00, 1e+00]
  QRHS range       [1e+00, 1e+00]
Presolve removed 45 rows and 42 columns

Continuous model is non-convex -- solving as a MI

[ Info: [🎉 ] defining the variables
[ Info: [🎣 ] imposing pattern on effective index set of λ
[ Info: [💣 ] imposing bound on the the entries of G, F based on Lyapunov function choice
[ Info: [🎍 ] adding the data generator function to create 𝐱, 𝐠, 𝐟
[ Info: [🎢 ] adding the constraints
[ Info: coninterpol ℓ = i_j_idx(0, 1)
[ Info: coninterpol ℓ = i_j_idx(1, 0)
[ Info: [🎇 ] adding objective
[ Info: [👲 ] warm-start values for all the variables
[ Info: [🙌 	🙏 ] model building done, starting the optimization process
[ Info: [🎴 ] Analyzing PRP/FR method
[ Info: [😻 ] optimal solution found done, store the solution
[ Info: [💹 ] warm-start objective value = 1.1249999794663557, and objective value of found solution = 1.1249999794663557
[ Info: IMPORTANT INFO
[ Info: ------------------------------------------
[ Info: ||g_{k-1}||^2 =  1.0916394762538901
[ Info: ||p_{k-1}||^2 = 2.1832789525077803
[ Info: NCG-PEP Theory {||p_k||^2/||g_k||^2} = 1.125
[ Info: NCG-PEP Numerical {||p_k||^2/||g_k||^2}= 1.1

([3.5242248760390797 1.95703488587636 1.3993761399899427 1.8463623918617562; 1.95703488587636 1.0916394762538901 0.7387961408053838 1.0916394762538901; 1.3993761399899427 0.7387961408053838 1.0 0.0; 1.8463623918617562 1.0916394762538901 0.0 2.1832789525077803], [1.0303357359923555, 0.6209709436660351], [1.8772918995294972 0.0 0.0 0.0; 1.0424776702902954 0.06985544502760473 0.0 0.0; 0.7454227764689438 -0.5481384962068047 0.37932713229805914 0.0; 0.9835244014660189 0.9496360593468655 -0.5604909073167766 0.0], 0.7499999793559416, 0.2392766704360783, 1.1249999794663557, Any[], Any[], 0.06755900382995605)